# 🎥 Colab Video Compressor

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AdittyaMondal/ColabVideoCompressor/blob/main/colab_notebook.ipynb)

Advanced video compression bot with GPU acceleration and multiple presets.

In [ ]:
#@title 🎥 Advanced Video Compressor Bot { display-mode: "form" }
#@markdown ### Configure and Run Your Video Compression Bot

import os
from IPython.display import HTML, display

#@markdown ## 1. Basic Configuration
APP_ID = ""  #@param {type:"string"}
API_HASH = ""  #@param {type:"string"}
BOT_TOKEN = "" #@param {type:"string"}
OWNER = "" #@param {type:"string"}
THUMBNAIL = "https://graph.org/file/75ee20ec8d8c8bba84f02.jpg" #@param {type:"string"}

#@markdown ## 2. Select Compression Preset
preset_option = "Balanced (Recommended)" #@param ["Ultra Fast (Lowest Quality)", "Balanced (Recommended)", "High Quality", "Max Quality (Slow)", "Custom"]

# Define presets dictionary with GPU acceleration (hwaccel flag added)
FFMPEG_PRESETS = {
    "Ultra Fast (Lowest Quality)": 'ffmpeg -y -hide_banner -loglevel error -hwaccel cuda -i "{}" -c:v hevc_nvenc -preset fast -rc constqp -qp 35 -c:a aac -b:a 128k -movflags +faststart "{}"',
    "Balanced (Recommended)": 'ffmpeg -y -hide_banner -loglevel error -hwaccel cuda -i "{}" -c:v hevc_nvenc -preset medium -rc constqp -qp 28 -c:a aac -b:a 192k -movflags +faststart "{}"',
    "High Quality": 'ffmpeg -y -hide_banner -loglevel error -hwaccel cuda -i "{}" -c:v hevc_nvenc -preset slow -rc constqp -qp 23 -c:a aac -b:a 256k -movflags +faststart "{}"',
    "Max Quality (Slow)": 'ffmpeg -y -hide_banner -loglevel error -hwaccel cuda -i "{}" -c:v hevc_nvenc -preset hq -rc constqp -qp 18 -c:a aac -b:a 320k -movflags +faststart "{}"'
}

#@markdown ### 3. Custom Preset Options (if selected above)
if preset_option == "Custom":
    custom_ffmpeg_command = 'ffmpeg -y -hide_banner -loglevel error -hwaccel cuda -i "{}" -c:v hevc_nvenc -preset medium -rc constqp -qp 28 -c:a aac -b:a 128k "{}"' #@param {type:"string"}
    FFMPEG_PRESETS["Custom"] = custom_ffmpeg_command

#@markdown ### 4. Advanced Settings
MAX_FILE_SIZE = 2000 #@param {type:"integer"}
MAX_QUEUE_SIZE = 10 #@param {type:"integer"}

def setup_bot():
    print("🚀 Starting bot setup...")

    if not all([APP_ID, API_HASH, BOT_TOKEN, OWNER]):
        raise ValueError("❌ All fields in 'Basic Configuration' are required!")

    print("Cloning repository...")
    !git clone -q https://github.com/AdittyaMondal/ColabVideoCompressor.git
    %cd ColabVideoCompressor

    print("Installing system packages...")
    !apt-get -qq update
    !apt-get -qq install -y ffmpeg mediainfo

    print("Creating directories...")
    !mkdir -p encode downloads thumb
    !chmod -R 777 encode downloads thumb

    print("Configuring environment...")
    ffmpeg_command = FFMPEG_PRESETS[preset_option].replace('"', '\"') # Escape quotes for the env file
    env_content = f"""APP_ID={APP_ID}
API_HASH={API_HASH}
BOT_TOKEN={BOT_TOKEN}
OWNER={OWNER}
THUMBNAIL='{THUMBNAIL}'
FFMPEG='{ffmpeg_command}'
MAX_FILE_SIZE={MAX_FILE_SIZE}
MAX_QUEUE_SIZE={MAX_QUEUE_SIZE}"""

    with open('.env', 'w') as f:
        f.write(env_content)

    print("Installing Python dependencies...")
    !pip install -q -r requirements.txt

    print("🤖 Starting bot...")
    !python3 -m bot

#@markdown ## 5. Run Setup and Start Bot
run_setup = True #@param {type:"boolean"}

if run_setup:
    try:
        setup_bot()
    except Exception as e:
        print(f"❌ An error occurred during setup: {str(e)}")

display(HTML('''
<div style='background-color: #f0f8ff; padding: 15px; border-radius: 8px; border: 1px solid #d1e7fd; margin-top: 20px;'>
<h3>📋 Important Notes:</h3>
<ul>
    <li><strong>Credentials:</strong> Get your <code>APP_ID</code> and <code>API_HASH</code> from <a href='https://my.telegram.org' target='_blank'>my.telegram.org</a>.</li>
    <li><strong>BOT_TOKEN:</strong> Get from @BotFather on Telegram.</li>
    <li><strong>OWNER ID:</strong> Get from @userinfobot (just send /start to it).</li>
    <li><strong>GPU Acceleration:</strong> All presets are now correctly configured to use Colab's GPU for both decoding and encoding.</li>
    <li><strong>Issues?</strong> If the bot stops, you may need to re-run this cell. Colab runtimes have time limits.</li>
</ul>
</div>
'''))